# 3. Processamento e Pré-processamento de Dados Geoespaciais
Nesta seção, vamos explorar operações fundamentais de pré-processamento de dados geoespaciais, essenciais para preparar os dados para análises e modelagens mais avançadas. Cada tópico será acompanhado de uma aplicação prática.

### 3.1 Geocoding e Geocoding Reverso
O geocoding é o processo de converter um endereço (como "Cristo Redentor, Rio de Janeiro") em coordenadas geográficas (latitude e longitude). Já o geocoding reverso realiza o caminho oposto: ele transforma coordenadas em um endereço textual.

##### Geocoding: endereço -> coordenadas

In [ ]:
from geopy.geocoders import Nominatim # Servico de Geocoding fornecido pelo OpenSteetMap

geolocator = Nominatim(user_agent="geoapi") # 
location = geolocator.geocode("Cristo Redentor, Rio de Janeiro")

print("Coordenadas do Cristo Redentor:")
print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")


#### Geocoding Reverso: coordenadas -> endereço

In [ ]:
reverse_location = geolocator.reverse((location.latitude, location.longitude), language="pt")
print("\nEndereço reverso aproximado:")
print(reverse_location.address)


#### Depois de converter um endereço em coordenadas geográficas (geocoding),podemos plotar o ponto em mapa com `folium`.

In [ ]:

m = folium.Map(location=[location.latitude, location.longitude], zoom_start=15)
folium.Marker([location.latitude, location.longitude], popup='Cristo Redentor').add_to(m)
m

Nesta seção, vamos transformar endereços de imóveis em coordenadas geográficas (latitude/longitude), e utilizar esses dados para fazer análises espaciais de preços.

O dataset contém informações sobre revenda de apartamentos públicos (HDB flats) em Singapura. Vamos:

- 🧭 Geocodificar os endereços
- 📍 Mapear os imóveis no espaço
- 📊 Analisar a variação de preços por localização

Este script abaixo automatiza o geocoding de endereços no dataset de imóveis sg-resale-flat-prices-2017-onwards.csv, transformando cada endereço em coordenadas geográficas (latitude e longitude).

Ele eh necessario pois como a API do OpenStreetMap limita 60 requisicoes por minuto, eh necessario pre-processar e criar o dataset em que faremos as analises.


In [ ]:
import pandas as pd
import time
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

rodar = False
if rodar:
    # Carregar dataset original
    df = pd.read_csv("datasets/Singapore/sg-resale-flat-prices-2017-onwards.csv")
    df["endereco"] = df["block"] + " " + df["street_name"] + ", " + df["town"] + ", Singapore"
    
    # Geocodificador
    geolocator = Nominatim(user_agent="geoapi_sg_full")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    
    # Tentar carregar resultados anteriores
    try:
        df_geo = pd.read_csv("datasets/Singapore/geocodificados.csv")
        processados = set(df_geo["endereco"])
        print(f"Retomando, {len(processados)} endereços já processados.")
    except FileNotFoundError:
        df_geo = pd.DataFrame()
        processados = set()
    
    novos = []
    batch_size = 50
    pause_seconds = 60
    
    # Iniciar processamento por blocos
    for i, row in df.iterrows():
        endereco = row["endereco"]
        if endereco in processados:
            continue
        
        try:
            location = geocode(endereco)
            lat = location.latitude if location else None
            lon = location.longitude if location else None
        except:
            lat, lon = None, None
    
        # Copiar a linha original e adicionar coordenadas
        nova_linha = row.copy()
        nova_linha["latitude"] = lat
        nova_linha["longitude"] = lon
        novos.append(nova_linha)
    
        print(f"{len(novos)} -> {endereco} => ({lat}, {lon})")
    
        if len(novos) % batch_size == 0:
            df_novos = pd.DataFrame(novos)
            df_geo = pd.concat([df_geo, df_novos], ignore_index=True).drop_duplicates(subset=["endereco"])
            df_geo.to_csv("datasets/Singapore/geocodificados.csv", index=False)
            print(f"📝 Salvo após {len(novos)} registros.")
            novos.clear()
            print(f"⏸️ Pausando por {pause_seconds} segundos...")
            time.sleep(pause_seconds)
    
    # Salvar o restante
    if novos:
        df_novos = pd.DataFrame(novos)
        df_geo = pd.concat([df_geo, df_novos], ignore_index=True).drop_duplicates(subset=["endereco"])
        df_geo.to_csv("datasets/Singapore/geocodificados.csv", index=False)
        print("✅ Processamento finalizado.")
    

In [ ]:
# Carregar os dados ja pre-processados 
df = pd.read_csv("datasets/Singapore/geocodificados.csv")

# Amostrar para testes rápidos
df_amostra = df.sample(n=30, random_state=42).copy()
df = df.dropna()

####  Mapa Interativo: Distribuição Espacial dos Preços por m²

Este mapa mostra a localização dos imóveis do programa HDB (Habitação Pública de Singapura) com base no dataset de revenda.

Cada ponto representa um imóvel geocodificado, e sua **cor** reflete o **preço por metro quadrado (preço/m²)**, calculado como:

Para classificar os imóveis, usamos uma abordagem **estatística baseada nos quartis**:

| Faixa | Critério | Cor |
|-------|----------|-----|
| **Barato** | Preço/m² abaixo do 1º quartil (Q1) | 🟢 Verde |
| **Na média** | Entre Q1 e Q3 (mediana) | 🟠 Laranja |
| **Caro** | Acima do 3º quartil (Q3) | 🔴 Vermelho |

In [ ]:
import folium

# Calcular o preço por m²
df["preco_m2"] = df["resale_price"] / df["floor_area_sqm"]

# Calcular quartis
q1 = df["preco_m2"].quantile(0.25)
q3 = df["preco_m2"].quantile(0.75)

# Função de cor baseada nos quartis
def cor_por_preco_m2(valor):
    if valor < q1:
        return "green"
    elif valor > q3:
        return "red"
    else:
        return "orange"

# Criar o mapa
m = folium.Map(location=[1.3521, 103.8198], zoom_start=12, tiles="CartoDB positron")

# Adicionar marcadores com cores baseadas em preço/m²
for _, row in df.iterrows():
    preco_total = row["resale_price"]
    preco_m2 = row["preco_m2"]
    popup_text = (
        f"<b>Preço total:</b> ${preco_total:,.0f}<br>"
        f"<b>Área:</b> {row['floor_area_sqm']} m²<br>"
        f"<b>Preço/m²:</b> ${preco_m2:,.0f}<br>"
        f"<b>Tipo:</b> {row['flat_type']}"
    )
    folium.CircleMarker(
        location=(row["latitude"], row["longitude"]),
        radius=6,
        color=cor_por_preco_m2(preco_m2),
        fill=True,
        fill_opacity=0.75,
        popup=folium.Popup(popup_text, max_width=250)
    ).add_to(m)

print(f"{len(df)} imóveis plotados com cores baseadas em preço/m².")
m


### ✅ Conclusão

Com base apenas nos endereços textuais, conseguimos:

- Localizar imóveis no mapa
- Visualizar a distribuição espacial dos preços
- Preparar os dados para análises como clusters, interpolações ou dashboards

Isso mostra como a **geocodificação é uma ponte entre dados tradicionais e análises espaciais**.


### 3.2 Projeções Cartográficas

Projeções cartográficas são transformações matemáticas que convertem a superfície curva da Terra (esferoide) em uma superfície plana (mapa). Como não é possível representar perfeitamente uma esfera em um plano, cada projeção faz **compromissos entre preservar forma, área, distância ou direção**.

#### 🔍 Por que isso importa?

Para muitas análises espaciais — como **cálculo de área, distância entre pontos, buffers e rotas** — é essencial que os dados estejam em uma **projeção apropriada**. A maioria dos dados geográficos brutos vem no sistema **WGS84** (EPSG:4326), que usa latitude e longitude (graus), mas não é adequado para medições métricas.

#### 🌎 Exemplos de Projeções

- **WGS84 (EPSG:4326)**: padrão global, usa graus. Bom para visualização, ruim para cálculos.
- **UTM (Universal Transverse Mercator)**: divide o globo em zonas com projeção métrica, ideal para cálculos de distância e área em regiões menores.
- **Albers Equal Area**: preserva área, útil para análise de distribuição espacial em grandes regiões.


**Aplicação:** Transformar um dataset de coordenadas geográficas (WGS84) para projeção métrica (UTM), facilitando o cálculo de distâncias reais.

In [ ]:
import geopandas as gpd

# Carregando shapefile dos municípios do RJ
shapefile_path = "datasets/RJ_2023/RJ_Municipios_2023.shp"
gdf = gpd.read_file(shapefile_path)

# Selecionar apenas um município (por exemplo, Rio de Janeiro)
rio = gdf[gdf["NM_MUN"] == "Rio de Janeiro"]

# Calcular área no sistema WGS84 (em graus) – NÃO CONFIÁVEL!
area_wgs84 = rio.geometry.area.iloc[0]

# Agora convertemos para UTM zona 23S (projeção métrica adequada)
rio_utm = rio.to_crs(epsg=31983)

# Calcular área no sistema UTM (em metros quadrados) – CONFIÁVEL!
area_utm = rio_utm.geometry.area.iloc[0]

# Mostrar os resultados
print(f"Área no sistema WGS84 (graus²): {area_wgs84}")
print(f"Área no sistema UTM (m²): {area_utm:,.2f}")
print(f"Área no sistema UTM (km²): {area_utm/1e6:,.2f}")


#### ✅ Conclusão:
- O valor em graus quadrados não tem interpretação prática direta — ele depende da curvatura da Terra e da latitude!

- Já o valor em metros quadrados, obtido com uma projeção como UTM, é o que você precisa para:

  - Calcular densidade populacional

  - Estimar área útil de uma zona

  - Fazer modelagem espacial confiável

🔗 Se você errar a projeção, você compromete os resultados do modelo!

### 3.3 Interseções de Camadas Geoespaciais

Uma das operações mais poderosas em ciência de dados geoespaciais é a **interseção entre camadas (layers)**. Essa técnica permite responder a perguntas espaciais complexas, como:

- 🏥 Quais hospitais estão em áreas de risco ambiental?
- 🌳 Quais bairros têm escolas dentro de zonas verdes?
- 🚨 Quais ocorrências policiais aconteceram dentro de determinada região?

Essas respostas emergem ao **cruzar diferentes fontes de dados espaciais** — como a sobreposição de **pontos, linhas e polígonos** com camadas raster (como imagens de satélite ou dados climáticos).

---

#### ⚡ Substituição de Usinas Fósseis por Energia Solar: Análise Espacial

Neste notebook, realizamos uma análise baseada em dados reais para avaliar o **potencial de substituição de usinas fósseis por energia solar no Brasil**.

A lógica é:
- Ver **onde as usinas fósseis estão localizadas**
- Verificar se essas regiões têm **alto potencial solar (PVOUT)**
- Calcular se seria **energeticamente viável substituir** a geração dessas usinas por usinas solares

In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Carregar dados das usinas fósseis no Brasil
usinas = pd.read_csv('datasets/solar_potential/globalpowerplantdatabasev130/global_power_plant_database.csv')
usinas_br = usinas[(usinas.country_long == "Brazil") & 
                   (usinas.primary_fuel.isin(["Coal", "Gas", "Oil"]))]

# Converter em GeoDataFrame
geometry = [Point(xy) for xy in zip(usinas_br.longitude, usinas_br.latitude)]
gdf = gpd.GeoDataFrame(usinas_br, geometry=geometry, crs="EPSG:4326")

# Carregar raster PVOUT
raster = rasterio.open("datasets\solar_potential\World_PVOUT_GISdata_LTAy_AvgDailyTotals_GlobalSolarAtlas-v2_GEOTIFF\World_PVOUT_GISdata_LTAy_DailySum_GlobalSolarAtlas_GEOTIFF\PVOUT.tif")

# Extrair valores de PVOUT para cada usina
coords = [(geom.x, geom.y) for geom in gdf.geometry]
pvout_values = [val[0] for val in raster.sample(coords)]
gdf["pvout"] = pvout_values

# Filtrar por alto potencial solar
gdf_altas = gdf[gdf["pvout"] > 4.6]  # limite ajustável
print(gdf_altas.shape)
print(gdf_altas.head())

# Visualização
fig, ax = plt.subplots(figsize=(10, 8))
gdf.plot(ax=ax, color="gray", label="Usinas fósseis")
gdf_altas.plot(ax=ax, color="orange", label="Alto potencial solar")
plt.title("Usinas fósseis em regiões com alto potencial de energia solar (PVOUT)")
plt.legend()
plt.grid(True)
plt.show()

#### 3.4 Interpolação Espacial (Kriging)

### 📌 O problema

Imagine que você possui medições de temperatura do fundo do mar feitas por sensores em pontos esparsos. Você quer criar um **mapa contínuo** para visualizar a variação da temperatura ao longo de toda a região.

Para isso, usamos **interpolação espacial** — uma técnica que estima valores em locais onde não há dados, com base na localização e valor dos pontos conhecidos.


### 🔄 Comparando abordagens

| Método | Como funciona | Limitações |
|--------|----------------|-------------|
| **IDW** (Inverse Distance Weighting) | A média ponderada dos pontos vizinhos, com pesos baseados na distância (“pontos mais próximos são mais parecidos”) | Não considera padrões espaciais complexos, só a distância |
| **Kriging** | Ajusta um modelo estatístico (variograma) para estimar valores com base na **estrutura espacial do dado** | Mais complexo, exige ajuste de modelo e interpretação mais técnica |


### 🧠 O que é o Kriging?

Kriging é uma técnica de interpolação **baseada em modelos estatísticos** que incorporam:

- A **distância** entre os pontos
- A **variabilidade dos dados**
- A **direção ou tendência** dos dados (em versões mais avançadas)

Ideal para fenômenos naturais como temperatura, poluição, teor de minério, etc.


In [ ]:
from pykrige.ok import OrdinaryKriging
from scipy.interpolate import griddata
# Recarregar o CSV pulando a linha com as unidades
df = pd.read_csv("datasets/temp_sea_bottom/northeast_atlatic_sea_bottom_temp.csv")
# Garantir conversão correta para tipos numéricos
df["latitude"] = pd.to_numeric(df["latitude"], errors="coerce")
df["longitude"] = pd.to_numeric(df["longitude"], errors="coerce")
df["sea_bottom_temperature"] = pd.to_numeric(df["sea_bottom_temperature"], errors="coerce")

# Remover linhas com valores faltantes
df = df.dropna(subset=["latitude", "longitude", "sea_bottom_temperature"])

# Exibir preview
df.head()


In [ ]:

# Amostrar 500 pontos para agilizar a krigagem
df_sample = df.sample(n=1000, random_state=42)

# Extrair variáveis
x = df_sample["longitude"].values
y = df_sample["latitude"].values
z = df_sample["sea_bottom_temperature"].values


In [ ]:
# Interpolação com IDW usando griddata
grid_x, grid_y = np.meshgrid(
    np.linspace(x.min(), x.max(), 100),
    np.linspace(y.min(), y.max(), 100)
)

zi_idw = griddata((x, y), z, (grid_x, grid_y), method='linear')


In [ ]:
# Interpolação com Kriging
OK = OrdinaryKriging(x, y, z, variogram_model='linear', verbose=False, enable_plotting=False)
zi_krig, _ = OK.execute("grid", grid_x[0], grid_y[:, 0])


In [ ]:
# Comparação visual
fig, axs = plt.subplots(1, 2, figsize=(18, 7))

# IDW
im1 = axs[0].imshow(zi_idw, extent=(x.min(), x.max(), y.min(), y.max()), origin='lower', cmap='coolwarm')
axs[0].set_title("Interpolação por IDW")
axs[0].scatter(x, y, c=z, edgecolors='k', s=10)
plt.colorbar(im1, ax=axs[0], label="Temperatura (°C)")

# Kriging
im2 = axs[1].imshow(zi_krig, extent=(x.min(), x.max(), y.min(), y.max()), origin='lower', cmap='coolwarm')
axs[1].set_title("Interpolação por Kriging")
axs[1].scatter(x, y, c=z, edgecolors='k', s=10)
plt.colorbar(im2, ax=axs[1], label="Temperatura (°C)")

plt.suptitle("Comparação: IDW vs Kriging", fontsize=16)
plt.tight_layout()
plt.show()
